# BERT Model with FAISS

In [ ]:
import pandas as pd
import re
import string
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import tensorflow
import sklearn
from tqdm import tqdm
from prettytable import PrettyTable

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Import the highest scored answers and accepted answers

In [ ]:
accepted_answers_df = pd.read_csv('Accepted_Answers.csv')
accepted_answers_df.head()

,Question,Relevant_Answer,question_id
0,InfoBar Component not visible after rendering,<p>You don't appear to be returning anything f...,73279519
1,Writing bits (not hex values) to file in Linux,"<p>perl to the rescue:</p>\n<pre class=""lang-b...",73414802
2,Change value of outerscope variable inside a f...,<p>Your variables are mutated from inside of a...,73370188
3,SmartContract/Solidity: The transaction has be...,<p><code>attributes[tokenId].lastClaimDate</co...,73364556
4,This name error keeps popping up despite me im...,<p>Conn is not defined here. Either change <co...,73417861


Import the questions and highest scored answers

In [ ]:
scored_answer_df = pd.read_csv('Scored_Answers.csv')
scored_answer_df.head()

,Question,Relevant_Answer,question_id
0,in android how to send and receive special sms...,<p>For this you have to create your custom bro...,29165268
1,Python Salary Program Using Conditionals,<p>You're using full quotes where you should b...,29089187
2,How can I show a spinner in a ViewPager when t...,<p>I would do a check for this inside the <cod...,29169909
3,Android: Horizontal list view on custom adaptor,<blockquote>\n <p>layout whose one item cover...,28878917
4,"Even If Testcase failed It is going in ""onTest...",<p>Function <code>verify</code> won't make a t...,29300800


In [ ]:
repeated_ids = scored_answer_df['question_id'].isin(accepted_answers_df['question_id']).any()

if repeated_ids:
    print("There are repeated ids in df2")
else:
    print("There are no repeated ids in df2")

There are no repeated ids in df2


Combine the two dataframes to create dataframe with questions and their relevant answer

In [ ]:
ground_truth_df = pd.concat([accepted_answers_df, scored_answer_df], axis=0, ignore_index=True)
ground_truth_df.head()

,Question,Relevant_Answer,question_id
0,InfoBar Component not visible after rendering,<p>You don't appear to be returning anything f...,73279519
1,Writing bits (not hex values) to file in Linux,"<p>perl to the rescue:</p>\n<pre class=""lang-b...",73414802
2,Change value of outerscope variable inside a f...,<p>Your variables are mutated from inside of a...,73370188
3,SmartContract/Solidity: The transaction has be...,<p><code>attributes[tokenId].lastClaimDate</co...,73364556
4,This name error keeps popping up despite me im...,<p>Conn is not defined here. Either change <co...,73417861


## Functions to clean the answers column

In [ ]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_punctuation_and_lower(text):
    text = text.lower()
    return text.translate(str.maketrans(' ', ' ', string.punctuation))

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    return [word for word in word_tokens if word not in stop_words]

def remove_usernames(text: str) -> str:
    """
    Remove usernames starting with '@' from the given text.

    Args:
        text (str): The input text containing usernames.

    Returns:
        str: The text with usernames removed.
    """
    cleaned_text = re.sub(r"@[A-Za-z0-9_]+", "", text)
    return cleaned_text

def clean_text(text):
    text = remove_html_tags(text)
    text = remove_usernames(text)
    text = remove_urls(text)
    text = remove_punctuation_and_lower(text)
    words = remove_stopwords(text)
    return ' '.join(words)


## Get the data of the questions and all the answers for each question(includes accepted answer)

In [ ]:
all_answers_df = pd.read_csv('All_answers.csv')
all_answers_df.head()

,question_id,question_title,answer_id,answer_body
0,930,How do I connect to a database and loop over a...,951,<p>@Goyuix -- that's excellent for something w...
1,930,How do I connect to a database and loop over a...,65774,<p>If you are intending on reading a large num...
2,930,How do I connect to a database and loop over a...,20049,<p>If you are querying a SQL Server database (...
3,930,How do I connect to a database and loop over a...,940,<p>Very roughly and from memory since I don't ...
4,930,How do I connect to a database and loop over a...,1022,<p>That's definitely a good way to do it. But...


#### Group the same questions and make a list of all answers for each question

In [ ]:
import pandas as pd

# Group the rows by question_id and aggregate the other columns into lists
grouped_answers_df = all_answers_df.groupby('question_id').agg({
    'question_title': lambda x: list(x)[0],
    'answer_id': lambda x: list(x),
    'answer_body': lambda x: list(x),
})

# Reset the index and rename the columns
grouped_answers_df = grouped_answers_df.reset_index().rename(columns={
    'answer_id': 'answer_ids',
    'answer_body': 'answer_bodies',
})


In [ ]:
grouped_answers_df

,question_id,question_title,answer_ids,answer_bodies
0,930,How do I connect to a database and loop over a...,"[951, 65774, 20049, 940, 1022, 2369, 56179247,...",[<p>@Goyuix -- that's excellent for something ...
1,22979,Detecting if SQL server is running,"[451160, 23019, 23011, 23010, 22991, 4217054]",[<p><code>System.Data.Sql.SqlDataSourceEnumera...
2,23027,CruiseControl.Net Build Publisher - Only publi...,"[32667, 32681, 23060, 7028931]",[<p>The default build publisher in CC.NET does...
3,34896,When is it best to sanitize user input?,"[34906, 34905, 34984, 34902, 36088, 34923, 349...",[<p>Users are evil!</p>\n\n<p>Well perhaps not...
4,37263,"Where does ""Change Management"" end and ""Projec...","[43187, 37565, 37277, 37291, 37292]","[<p><strong>What is change management, and how..."
...,...,...,...,...
19995,73829515,Big Query first non null value record type,"[73829947, 73830169]",[<p>You may consider below scalar subquery for...
19996,73829809,Change Wordpress colors based on logo main color,[73829900],"[<p>First of all, you need colors from the ima..."
19997,73831805,Why are methods not valid constructors?,[73832791],[<p>As pointed out in the comments on the orig...
19998,73834279,No ability to publish newer build for external...,[73834304],[<p>I believe that there is possible to do mor...


## Create input pairs of question and answer along with the labels.

In [ ]:
## Using unprocessed data to create question and answer pairs

from sklearn.model_selection import train_test_split

# Create question-answer pairs and labels
input_pairs = []
labels = []

questions_list = grouped_answers_df['question_title'].values
answers_list = grouped_answers_df['answer_bodies'].values
correct_answers_list = ground_truth_df['Relevant_Answer'].values

for question, ans_list in zip(questions_list, answers_list):
    for answer in ans_list:
        input_pairs.append((question, answer))
        labels.append(1 if answer in correct_answers_list else 0)


# Split data into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(input_pairs, labels, test_size=0.3, random_state=42, stratify=labels)


# Transformer model

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.4 MB/s eta 0:00:00


## BERT model and tokenizer

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support



In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length = 512

In [ ]:
def tokenize_and_encode(data, max_length=248):
    return tokenizer.batch_encode_plus(data, truncation=True, padding=True, max_length=max_length, return_tensors='pt')

train_encodings = tokenize_and_encode(train_data)
val_encodings = tokenize_and_encode(val_data)

In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
train_encodings['input_ids'][0]

tensor([  101, 16246, 26899, 10938,  1024,  2129,  2000,  2224, 21461,  3215,
         4048,  6199, 11178,  2007, 21461,  2102,   102,  1026,  1052,  1028,
         1996,  6210,  2005,  1996,  6434,  1997,  1026,  3642,  1028, 21461,
         2102,  1026,  1013,  3642,  1028,  1006,  1998,  1026,  3642,  1028,
         2065,  6199,  1026,  1013,  3642,  1028,  1007,  2003,  2182,  1024,
         1026,  1037, 17850, 12879,  1027,  1000,  8299,  1024,  1013,  1013,
         9986,  2015,  1012, 16596,  7685,  1012,  8917,  1013,  9986,  1013,
        16371,  8737,  2100,  1013,  4431,  1013, 23964,  1012, 21461,  2102,
         1012, 16129,  1001,  4281,  1011,  2592,  1000,  2128,  2140,  1027,
         1000,  2053, 14876,  7174,  2860,  1000,  1028,  8299,  1024,  1013,
         1013,  9986,  2015,  1012, 16596,  7685,  1012,  8917,  1013,  9986,
         1013, 16371,  8737,  2100,  1013,  4431,  1013, 23964,  1012, 21461,
         2102,  1012, 16129,  1001,  4281,  1011,  2592,  1026, 

In [ ]:
# visualizing the tokens
tokens = tokenizer.convert_ids_to_tokens(train_encodings['input_ids'][0])
for token, id in zip(tokens, train_encodings['input_ids'][0]):
    print('{:8}{:8,}'.format(token,id))

[CLS]        101
discrete  16,246
fourier   26,899
transform  10,938
:          1,024
how        2,129
to         2,000
use        2,224
ff        21,461
##ts       3,215
##hi       4,048
##ft       6,199
correctly  11,178
with       2,007
ff        21,461
##t        2,102
[SEP]        102
<          1,026
p          1,052
>          1,028
the        1,996
definition   6,210
for        2,005
the        1,996
output     6,434
of         1,997
<          1,026
code       3,642
>          1,028
ff        21,461
##t        2,102
<          1,026
/          1,013
code       3,642
>          1,028
(          1,006
and        1,998
<          1,026
code       3,642
>          1,028
if         2,065
##ft       6,199
<          1,026
/          1,013
code       3,642
>          1,028
)          1,007
is         2,003
here       2,182
:          1,024
<          1,026
a          1,037
hr        17,850
##ef      12,879
=          1,027
"          1,000
http       8,299
:          1,024
/         

In [ ]:
# Visualizing the number of token in question and text
sep_idx = (train_encodings['input_ids'][0].tolist()).index(tokenizer.sep_token_id)
print("SEP token index: ", sep_idx)
num_seg_a = sep_idx + 1
print("Number of tokens in segment A (question): ", num_seg_a)
num_seg_b = len(train_encodings['input_ids'][0].tolist()) - num_seg_a
print("Number of tokens in segment B (answer): ", num_seg_b)

SEP token index:  16
Number of tokens in segment A (question):  17
Number of tokens in segment B (answer):  231


In [ ]:
train_encodings['token_type_ids'][0]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 0, 0, 0])

In [ ]:
train_encodings['attention_mask'][0]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 0, 0, 0])

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

In [ ]:
# checking the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Working on {device}')

Working on cuda


In [ ]:
class QADataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = QADataset(train_encodings, train_labels)
test_dataset = QADataset(val_encodings, val_labels)

In [ ]:
# Define the dataloaders
train_loader = DataLoader(train_dataset, batch_size=15, shuffle=True)
valid_loader = DataLoader(test_dataset, batch_size=15)

# Model evaluation before fine tuning

In [ ]:
model.to(device)
model.eval()

acc = []

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tqdm import tqdm

# Initialize empty lists for true labels and predictions
all_labels = []
all_predictions = []

model.eval()

for batch in tqdm(valid_loader):
    with torch.no_grad():
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)

        predictions = torch.argmax(outputs.logits, dim=1)

        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predictions.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')
conf_matrix = confusion_matrix(all_labels, all_predictions)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")
print(f"Confusion Matrix:\n{conf_matrix}")


  0%|          | 0/669 [00:00<?, ?it/s]<ipython-input-79-0515eb8a8bcf>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 669/669 [02:18<00:00,  4.83it/s]

Accuracy: 0.5284674444112075
Precision: 0.5152881993888415
Recall: 0.5284674444112075
F1-score: 0.5196738547413459
Confusion Matrix:
[[1333 2696]
 [2033 3967]]


# Fine tuning BERT

In [ ]:
# Fine tuning it per batch
N_EPOCHS = 2
optim = AdamW(model.parameters(), lr=2e-5,eps = 1e-8)

model.to(device)
model.train()

for epoch in range(N_EPOCHS):
  loop = tqdm(train_loader, leave=True)
  for batch in loop:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    label = batch['labels'].to(device)
    # model.zero_grad()
    outputs = model(input_ids, token_type_ids = None, attention_mask=attention_mask, labels=label)
    loss = outputs[0]
    loss.backward()
    optim.step()
    loop.set_description(f'Epoch {epoch+1}')
    loop.set_postfix(loss=loss.item())

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/1560 [00:00<?, ?it/s]<ipython-input-79-0515eb8a8bcf>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 2: 100%|██████████| 1560/1560 [16:32<00:00,  1.57it/s, loss=0.687]


In [ ]:
model.save_pretrained('/content/drive/MyDrive/SEM 3/Code/Saved_BERT_model')

In [ ]:
model.to(device)


acc = []

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tqdm import tqdm

# Initialize empty lists for true labels and predictions
all_labels = []
all_predictions = []

model.eval()

for batch in tqdm(valid_loader):
    with torch.no_grad():
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)

        predictions = torch.argmax(outputs.logits, dim=1)

        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predictions.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')
conf_matrix = confusion_matrix(all_labels, all_predictions)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")
print(f"Confusion Matrix:\n{conf_matrix}")


  0%|          | 0/669 [00:00<?, ?it/s]<ipython-input-79-0515eb8a8bcf>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 669/669 [02:19<00:00,  4.80it/s]


Accuracy: 0.6293748130421777
Precision: 0.6489271667641662
Recall: 0.6293748130421777
F1-score: 0.5469097122735136
Confusion Matrix:
[[ 576 3453]
 [ 264 5736]]


In [ ]:
loaded_model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/SEM 3/Code/Saved_BERT_model')
# loaded_model = BertForSequenceClassification.from_pretrained('/content/drive/Shareddrives/SEM 3/DATA298A  /Code/Saved_BERT_model')

In [ ]:
# Fine tuning it per batch ## 4 epochs
N_EPOCHS = 2
optim = AdamW(loaded_model.parameters(), lr=2e-5,eps = 1e-8)

loaded_model.to(device)
loaded_model.train()

for epoch in range(N_EPOCHS):
  loop = tqdm(train_loader, leave=True)
  for batch in loop:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    label = batch['labels'].to(device)
    # model.zero_grad()
    outputs = loaded_model(input_ids, token_type_ids = None, attention_mask=attention_mask, labels=label)
    loss = outputs[0]
    loss.backward()
    optim.step()
    loop.set_description(f'Epoch {epoch+1}')
    loop.set_postfix(loss=loss.item())

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/1560 [00:00<?, ?it/s]<ipython-input-79-0515eb8a8bcf>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 2: 100%|██████████| 1560/1560 [16:33<00:00,  1.57it/s, loss=0.396]


In [ ]:
## After 25 epochs

In [ ]:
loaded_model.to(device)

acc = []

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix,classification_report
from tqdm import tqdm

# Initialize empty lists for true labels and predictions
all_labels = []
all_predictions = []

loaded_model.eval()

for batch in tqdm(valid_loader):
    with torch.no_grad():
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = loaded_model(input_ids, attention_mask=attention_mask)

        predictions = torch.argmax(outputs.logits, dim=1)

        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predictions.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')
conf_matrix = confusion_matrix(all_labels, all_predictions)

# Generate classification report
report = classification_report(all_labels, all_predictions)

# Print the classification report
print(report)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")
print(f"Confusion Matrix:\n{conf_matrix}")


  0%|          | 0/669 [00:00<?, ?it/s]<ipython-input-21-0515eb8a8bcf>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 669/669 [02:25<00:00,  4.59it/s]


              precision    recall  f1-score   support

           0       0.51      0.41      0.45      4029
           1       0.65      0.74      0.69      6000

    accuracy                           0.61     10029
   macro avg       0.58      0.57      0.57     10029
weighted avg       0.59      0.61      0.60     10029

Accuracy: 0.6054442117858211
Precision: 0.5940329880625728
Recall: 0.6054442117858211
F1-score: 0.5956009693359
Confusion Matrix:
[[1637 2392]
 [1565 4435]]


## Create embeddings of all questions in dataframe

In [ ]:
ground_questions_list = ground_truth_df['Question'].values

In [ ]:
def encode(text, tokenizer, model, device):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors='pt').to(device)
    with torch.no_grad():
        output = model.bert(**inputs)
    return output.last_hidden_state[:, 0, :].cpu().numpy()


In [ ]:
question_embeddings = [encode(question, tokenizer, loaded_model,device) for question in ground_questions_list]

Save the embeddings of all questions

In [ ]:
question_embeddings = np.array(question_embeddings)

In [ ]:
np.save('question_embeddings.npy', question_embeddings)

## FAISS

In [ ]:
!pip install faiss-cpu  # for CPU version
# or
!pip install faiss-gpu  # for GPU version


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.1 MB/s eta 0:00:00


In [ ]:
import faiss
import numpy as np


In [ ]:
question_embeddings = np.load('question_embeddings.npy')

In [ ]:
# Convert question embeddings to a numpy matrix
question_embeddings_matrix = np.vstack(question_embeddings)

# Get the dimension of the embeddings
dimension = question_embeddings_matrix.shape[1]

# Create a FAISS index
index = faiss.IndexFlatL2(dimension)

# Add the question embeddings to the index
index.add(question_embeddings_matrix)


In [ ]:
question_embeddings_matrix.shape

(20000, 768)

In [ ]:
def find_top_similar_question_answer_indices(user_input, tokenizer, model, device, index, k=5):
    # Encode the user input
    user_input_embedding = encode(user_input, tokenizer, model, device)

    # Search the index for the top k similar embeddings
    _, indices = index.search(user_input_embedding, k)

    # Flatten the indices array
    indices = indices.flatten()

    return indices


## User input and create embeddings

In [ ]:
# checking the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Working on {device}')

Working on cuda


In [ ]:
loaded_model = BertForSequenceClassification.from_pretrained('Saved_BERT_model')

In [ ]:
loaded_model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
question_embeddings = np.load('question_embeddings.npy')

In [ ]:
# call big query for scores and answer bodies

all_answersscore_df = pd.read_csv('All_answers_scores.csv')
all_answersscore_df.head()

,question_id,question_title,answer_id,answer_body,score
0,13128,How can I combine several C/C++ libraries into...,27676078,"<p>On Linux, <a href=""https://en.wikipedia.org...",8
1,13128,How can I combine several C/C++ libraries into...,13136,<p>I'm not sure how to physically combine them...,0
2,13128,How can I combine several C/C++ libraries into...,22041,"<p>Maybe I'm misunderstanding, but don't you o...",0
3,13128,How can I combine several C/C++ libraries into...,13140,"<p>On Unix like systems, the ld and ar utiliti...",9
4,13128,How can I combine several C/C++ libraries into...,13206,<p>You could extract the object files from eac...,26


In [ ]:
def pretty_table(fetched_values):
  # Create a pretty table object
  table = PrettyTable()

  # Add a column header to the table
  table.field_names = ['Answer 1', 'Answer 2','Answer 3', 'Answer 4', 'Answer 5']

  for i in range(0,len(fetched_values)):
    answer = remove_html_tags(fetched_values[i])
    answer = remove_usernames(answer)
    answer = ''.join(answer)
    fetched_values[i] = answer

  table.add_row(fetched_values[:5])
  # Set alignment for each column to left
  for field in table.field_names:
    table.align[field] = 'l'

  # Print the pretty table
  print(table)


In [ ]:
def final_results(user_input):

  top_question_indices = find_top_similar_question_answer_indices(user_input, tokenizer, loaded_model, device, index)
  # print(top_question_indices)

  #### Groundtruth ranks
  ## initiate score dataframe

  scores_df = pd.DataFrame({'score':[], 'answer_id':[], 'answer_body' : []})
  scores_df['answer_id'] = scores_df['answer_id'].astype(int)

  ## get scores
  for ind in top_question_indices:
    if len(scores_df) < 5:
      q_id = ground_truth_df.loc[ind, 'question_id']
      filtered_df = all_answersscore_df[all_answersscore_df['question_id'] == q_id]
      # Select 'score','answr body' and 'answer_id' columns
      filtered_df = filtered_df[['score', 'answer_id','answer_body']]
      scores_df = pd.concat([scores_df,filtered_df], axis=0, ignore_index=True)
    else:
      break

  ## getting answer bodies
  scores_df.reset_index(drop=True, inplace=True)
  answers_text_list = scores_df['answer_body'].to_list()
  answers_embeddings = [encode(answer, tokenizer, loaded_model,device) for answer in answers_text_list]

  ## FAISS
  # Convert answer embeddings to a numpy matrix
  answer_embeddings_matrix = np.vstack(answers_embeddings)
  # Get the dimension of the embeddings
  dimension = answer_embeddings_matrix.shape[1]
  # Create a FAISS index
  answer_index = faiss.IndexFlatL2(dimension)
  # Add the question embeddings to the index
  answer_index.add(answer_embeddings_matrix)

  ## getting top answer indices
  top_answer_indices = find_top_similar_question_answer_indices(user_input, tokenizer, loaded_model, device, answer_index)



  fetched_values = []
  for i in top_answer_indices:
    answer_i = scores_df['answer_body'].iloc[i]
    fetched_values.append(answer_i)

  pretty_table(fetched_values)



In [ ]:
user_input = "Subversion Berkeley DB broken, recovery failed?"
final_results(user_input)

+-----------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Answer 1                                                                                | Answer 2                                                                                                                                                    

In [ ]:
user_input = "Subversion Berkeley DB broken, the recovery has failed. Why?"
final_results(user_input)

+-----------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Answer 1                                                                                | Answer 2                                                                                                                                                                          

In [ ]:
user_input = "How do I create collision detections for my bouncing balls?"
final_results(user_input)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------

In [ ]:
user_input = 'How can display driver version be obtained on the Mac?'
final_results(user_input)

+---------------------------------------------------------------------------------------+-------------------------------------------------------------------------------+-------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------+
| Answer 1                                                                              | Answer 2                                                                      | Answer 3                                                                            | Answer 4                                                                                                                                  